### Importamos las librerias necesarias para realizar nuestro ETL:

In [1]:
from pyspark.sql.functions import concat_ws, col, sum, from_unixtime
from pyspark.sql import SparkSession

### Creacion de tabla `processed_files` para mantener el registro de los archivos ya procesados.

In [3]:
spark.sql("CREATE TABLE IF NOT EXISTS processed_files_reviews_google (file_name STRING) USING DELTA")

### Obtenemos la lista de archivos contenidos en el storage. En este caso ADLS.

In [2]:
# Crear una instancia de SparkSession
spark = SparkSession.builder.getOrCreate()

# Especificar el nombre del contenedor y directorio
container_name = "datumtech"
directory_path = "/GoogleMaps/reviews-estados/"

# Obtener la lista de carpetas
adls_files = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration()) \
    .listStatus(spark._jvm.org.apache.hadoop.fs.Path(f"abfss://{container_name}.blob.core.windows.net/{directory_path}"))

### Creamos un variable `new_files` que contiene los archivos que no estan en ADLS.

In [3]:
processed_files = spark.sql("SELECT file_name FROM processed_files_reviews_google").toPandas()["file_name"].tolist()

new_files = [file.getPath().getName() for file in adls_files if file.getPath().getName() not in processed_files]

### Podemos ver que archivos ya estan procesados en la tabla `processed_files`

In [4]:
spark.sql("SELECT * FROM processed_files_reviews_google").show()

### Podemos ver que archivos no estan procesados aun.

In [5]:
for file in new_files:
    print(file)

### Creamos la funcion `etl` que realiza todo el proceso y devuelve el dataframe en formato parquet a la tabla silver corrrespondiente a los datos ya procesados.

In [6]:

def etl(file):
    
    # Definimos las rutas:
    path_raw=f"abfss://datumtech@datumlake.dfs.core.windows.net/GoogleMaps/reviews-estados/{file}"
    path_bronze = f"abfss://datumtech@datumlake.dfs.core.windows.net/bronze/GoogleMapsbronze/reviews-estados-bronze/{file}-bronze"
    path_silver = f"abfss://datumtech@datumlake.dfs.core.windows.net/silver/GoogleMapssilver/reviews-estados-silver/{file}-silver"

    # Cargamos el archivo desde ADLS. Nos quedamos solo con las columnas consideradas para el proyecto:
    df_raw = spark.read.format("json").load(path_raw).select('user_id', 'gmap_id', 'rating', 'text', 'time')
     
    # Guardamos el DataFrame df_raw en la tabla bronze correspondiente a los datos en crudo o poco procesados en Azure Data Lake con un formato parquet ideal para manejar altos volumenes de datos.
    df_raw.write.format("parquet").save(path_bronze)
        
    # Cargamos el archivo desde la tabla bronze en Azure Data Lake en un DataFrame.
    df_review = spark.read.format("parquet").load(path_bronze)

    # Eliminar los duplicados
    df_review = df_review.dropDuplicates()

    # Rellenamos valores vacíos o nulos en las columnas 'text', y 'rating'. A pesar de no tener nulos en algunas, dejamos planteado el codigo para usar el notebook en jobs posteriores.
    # Eliminamos los registros donde 'gmap_id', 'user_id' y 'time' son nulos o vacios. Representan menos del 10% de los datos totales.
    df_review = df_review.fillna('Unknown', subset=['text'])
    df_review = df_review.fillna(0, subset=['rating'])
    df_review = df_review.na.drop(subset=['user_id', 'gmap_id', 'time'])

    # Transformar la columna 'time' en formato de fecha
    df_review = df_review.withColumn("date", from_unixtime(df_review.time / 1000).cast("date"))
    df_review = df_review.drop('time')

    # Guardamos el DataFrame df_review en la tabla silver correspondiente a los datos procesados en Azure Data Lake.
    return df_review.write.format("parquet").save(path_silver)
    

### Iteramos sobre cada archivo sin procesar.

In [7]:
for file in new_files:
    etl(file)

### Agregamos a la tabla `processed_files_reviews_google` los archivos ya procesados.

In [8]:
new_files_df = spark.createDataFrame([(file,) for file in new_files], ["file_name"])
new_files_df.write.format("delta").mode("append").saveAsTable("processed_files_reviews_google")

### Podemos verificar que, efectivamente esten registrados los archivos ya procesados.

In [25]:
spark.sql("SELECT * FROM processed_files_reviews_google").show()

#### Para comprobar que todo se haya ejecutado de manera correcta, podemos traer cualquier archivo de la tabla silver y hacer algunas verificaciones.

In [16]:
# Funcion para cargar el archivo desde la tabla correspondiente al estado de los datos en Azure Data Lake en un DataFrame.
def load_from(file, level):
    path= f"abfss://datumtech@datumlake.dfs.core.windows.net/{level}/GoogleMaps{level}/reviews-estados-{level}/{file}-{level}"
    df = spark.read.format("parquet").load(path)
    return df

In [17]:
# Cambiando ...new_files[<valor de la fila en la tabla>][1]... podras cargar algun archivo de los ya procesados de la tabla silver.
df = load_from(new_files[1], "silver")

In [18]:
df.show()

In [19]:
df.count()

In [20]:
df.dtypes

### Podemos verificar si hay nulos en algun archivo en la tabla silver.

In [21]:
# Funcion para el conteo de nulos del dataframe.
def null_counts (df):
    counts = df.select([sum(col(c).isNull().cast("integer")).alias(c) for c in df.columns])
    return counts.show()

In [22]:
# Vemos que columnas poseen nulos y en que cantidad.
nulls = null_counts(df)